# Find Nearest Road Segment

In [1]:
import geopandas as gp
import pandas as pd
import os
import random
from shapely.geometry import Point, LineString
from scipy.signal import medfilt
from more_itertools import unique_everseen
import folium

pd.options.display.max_rows = 16

### Choose file to analyze

In [2]:
tripstart = 41200
filenum = 2
filename = 'TripStart_{0}_p{1:0>3}.csv'.format(tripstart,filenum)

### Read in road segment shapefile and extract spatial index

Shapefile for the Michigan road network can be downloaded from the **State of Michigan GIS Open Data** at http://gis-michigan.opendata.arcgis.com/. You can also get it in KML or GeoJSON.

In [3]:
%%time
roads = gp.read_file('MIroads.shp')
sidx = roads.sindex

Wall time: 2min 32s


### Read in BSM file

We round the lat/lon coordinates to 4 decimal places (~11m resolution) and eliminate duplicates. We will merge back the road names with the original dataframe to avoid looking up similar gps coordinates for the same road.

In [4]:
columns = 'RxDevice,FileId,TxDevice,Gentime,TxRandom,MsgCount,DSecond,Latitude,Longitude,Elevation,Speed,Heading,Ax,Ay,Az,Yawrate,PathCount,RadiusOfCurve,Confidence'
columns = columns.split(',')
trip = pd.read_csv(filename, header=None)
trip.columns = columns
gps4 = trip[[7,8]].round(4)
gps4.columns = ['lat','lon']
trip = pd.concat([trip,gps4], axis=1)
gps4.drop_duplicates(['lat','lon'], inplace=True)
gps4.reset_index(drop=True, inplace=True)


### Find closest road to gps point

In [5]:
%%time
radius = 0.0001
closest_road = []
gps = zip(gps4['lon'],gps4['lat'])
for i, xy in enumerate(gps):
    pt = Point(xy)
    road_id = list(sidx.nearest(pt.buffer(radius).bounds, 1))
    road_dist = [(id,roads.at[id,'geometry'].distance(pt)) for id in road_id]
    road_dist.sort(key=lambda x: x[1])
    closest_road.append(road_dist[0])

nearest_rd = pd.DataFrame(closest_road, columns=['rd1','d1'])
gpsrds = pd.concat([gps4, nearest_rd], axis=1)
# apply median filter to roads to try to remove spurious intersections
gpsrds['rd1'] = medfilt(gpsrds['rd1'],11)
trip_roads = trip.merge(gpsrds, how='left', on=['lat','lon'], suffixes=('','_y'))
trip_roads.drop(['lat','lon'], axis=1, inplace=True)
# merge it back with the original dataframe
trip_roadnames = trip_roads.merge(roads[['RDNAME']], how='left', left_on='rd1', right_index=True)

Wall time: 37.9 s


### Print out road names for each trip

In [6]:
list_trips = []
for (rx,id,tx),df in trip_roadnames.groupby(['RxDevice','FileId','TxDevice']):
    roadlist = df['RDNAME'].tolist()
    print('\nRxDevice={} FileId={} TxDevice={} went through these roads: '.format(rx,id,tx) )
    print(list(unique_everseen(df['RDNAME'].tolist())))
    # grab every 20th data point for the web map
    data = df.iloc[::20,[7,8]]
    list_trips.append(list(zip(data['Latitude'],data['Longitude'])))


RxDevice=22 FileId=22490 TxDevice=22 went through these roads: 
['Plymouth Rd', 'Green Rd', 'Whisperwood Dr', 'Nixon Rd', 'Dhu Varren Rd', 'N Foxridge Ct', 'Omlesaad Dr']

RxDevice=22 FileId=22491 TxDevice=22 went through these roads: 
['Green Rd', 'Commonwealth Blvd', 'Plymouth Rd', 'Georgetown Blvd', 'S Huron Pkwy', 'Nixon Rd', 'Traverwood Dr', 'Lake Lila Dr', 'Upland Dr', 'Barton Dr', None, 'Barton Shore Dr', 'Whitmore Lake Rd', 'W M 14', 'E M 14', 'Huron River Dr', 'Miller Rd', 'Dexter Ann Arbor Rd', 'Wagner Rd', 'W I 94', 'I 94 Crossover', 'Baker Rd', 'N Dancer Rd', 'Old US 12', 'Kalmbach Rd', 'Notten Rd', 'Mount Hope Rd', 'Whipple Rd', 'Sargent Rd', 'E I 94', 'Hawkins Rd', 'N Elm Ave', 'Blackman Rd', 'N Sandstone Rd', 'N Dearing Rd', 'W Michigan Ave', 'Eaton Rapids Rd', '29 Mile Rd', '28 Mile Rd', '27 Mile Rd', '24 Mile Rd', '23 Mile Rd', '22 1/2 Mile Rd', 'Partello Rd', '18 1/2 Mile Rd', '17 1/2 Mile Rd', 'Old US 27 N', 'Michigan Ave E', '9 Mile Rd', 'Beadle Lake Rd', '6 1/2 Mi

### Create a Leaflet map

In [7]:
trip_map = folium.Map([42.264768, -83.755503], zoom_start= 10)
colours = ['red','blue','green','orange','yellow','black','white','magenta','lightblue','lightgreen']
for route in list_trips:
    folium.PolyLine(route, color=random.choice(colours)).add_to(trip_map)

### Display map

In [8]:
trip_map